In [7]:
from scipy.signal import stft
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import os
import numpy as np
from scipy.optimize import curve_fit


In [ ]:
# 定义多项式函数
def fitting_model_func(x, a, b, c, d, e):
    return a * x**4 + b * x**3 + c * x**2 + d * x + e


# Convert quaternion to rotation matrix
def rotation_matrix(quat):
    rot_mat = np.ones([3,3])
    a = quat[0]**2
    b = quat[1]**2
    c = quat[2]**2
    d = quat[3]**2
    e = quat[0]*quat[1]
    f = quat[0]*quat[2]
    g = quat[0]*quat[3]
    h = quat[1]*quat[2]
    i = quat[1]*quat[3]
    j = quat[2]*quat[3]
    rot_mat[0,0] = a - b - c + d
    rot_mat[0,1] = 2 * (e - j)
    rot_mat[0,2] = 2 * (f + i)
    rot_mat[1,0] = 2 * (e + j)
    rot_mat[1,1] = -a + b - c + d
    rot_mat[1,2] = 2 * (h - g)
    rot_mat[2,0] = 2 * (f - i)
    rot_mat[2,1] = 2 * (h + g)
    rot_mat[2,2] = -a - b + c + d
    
    return rot_mat


# 创建旋转矩阵
def rpy2Rotation(roll, pitch, yaw):
    rotation_matrix = np.array([
        [np.cos(yaw)*np.cos(pitch), -np.sin(yaw)*np.cos(roll) + np.cos(yaw)*np.sin(pitch)*np.sin(roll), np.sin(yaw)*np.sin(roll) + np.cos(yaw)*np.sin(pitch)*np.cos(roll)],
        [np.sin(yaw)*np.cos(pitch), np.cos(yaw)*np.cos(roll) + np.sin(yaw)*np.sin(pitch)*np.sin(roll), -np.cos(yaw)*np.sin(roll) + np.sin(yaw)*np.sin(pitch)*np.cos(roll)],
        [-np.sin(pitch), np.cos(pitch)*np.sin(roll), np.cos(pitch)*np.cos(roll)]
    ])
    return rotation_matrix


def cfar_so(xc, LW_Size, Guard_Size, PFA):
    alpha = LW_Size * (PFA ** (-1 / LW_Size) - 1)
    target = []
    index = np.arange(1 + LW_Size/2 + Guard_Size/2, len(xc) - LW_Size/2 - Guard_Size/2)
    CFAR_Threshold = np.zeros(len(index))

    for i in index:
        cell_left = xc[int(i - LW_Size/2 - Guard_Size/2) : int(i - Guard_Size/2)]
        # cell_right = xc[int(i + Guard_Size/2 + 1) : int(i + N/2 + Guard_Size/2 + 1)]
        cell_right = 1e10
        Z = np.min([np.mean(cell_left), np.mean(cell_right)])
        # print(alpha)
        CFAR_Threshold[int(i - LW_Size/2 - Guard_Size/2-1)] = Z * alpha
        if xc[int(i)] > Z * alpha:
            target.append(int(i))
    return index, CFAR_Threshold, np.array(target)



In [8]:

    
def main():
    input_folder = "/home/nuci7/project/AirTouch/crazyflie_filter_constant/control/data/"
    output_folder = "/home/nuci7/project/AirTouch/crazyflie_filter_constant/control/temp4real/"
    imu_data = np.loadtxt(input_folder+"imu_data_0102-2131.csv", delimiter=',')
    
    # acc_x, acc_y, acc_z, gyro_x, gyro_y, gyro_z, motor1, motor2, motor3, motor4, quat, current_time, height
    # Normalizing the PWM
    imu_data[:,6:10] = imu_data[:,6:10]/((1<<16)-1)
    fs=100
    nperseg=200
    noverlap=199


    # 创建一个包含3行6列的子图布局
    fig, axes = plt.subplots(nrows=3, ncols=9, figsize=(28, 8))

    # 绘制第1个子图
    f, t, Zxx = stft(imu_data[:,0], fs=100, nperseg=nperseg, noverlap=noverlap)
    pcm = axes[0, 0].pcolormesh(t, f, np.abs(Zxx)**2, shading='auto')
    # cbar = fig.colorbar(pcm, ax=axes[0, 0], label='Amplitude')
    axes[0, 0].set_title('acc_x')
    pp = np.sum(np.abs(Zxx), axis=0)
    time_vector = np.arange(len(pp)) *((nperseg-noverlap)/fs)
    axes[1, 0].plot(time_vector, pp)
    axes[1, 0].set_title('acc_x')
    axes[2, 0].plot(time_vector, np.sum(abs(Zxx[12:17,:]**2),axis=0))
    axes[2, 0].set_title('acc_x')
    f1 = np.sum(abs(Zxx[12:17,:]**2),axis=0)


    # 绘制第2个子图
    f, t, Zxx = stft(imu_data[:,1], fs=100, nperseg=nperseg, noverlap=noverlap)
    pcm = axes[0, 1].pcolormesh(t, f, np.abs(Zxx)**2, shading='auto')
    # cbar = fig.colorbar(pcm, ax=axes[0, 0], label='Amplitude')
    axes[0, 1].set_title('acc_y')
    pp = np.sum(np.abs(Zxx), axis=0)
    time_vector = np.arange(len(pp)) *((nperseg-noverlap)/fs)
    axes[1, 1].plot(time_vector, pp)
    axes[1, 1].set_title('acc_y')
    axes[2, 1].plot(time_vector, np.sum(abs(Zxx[12:17,:]**2),axis=0))
    axes[2, 1].set_title('acc_y')
    f2 = np.sum(abs(Zxx[12:17,:]**2),axis=0)


    # 绘制第3个子图
    f, t, Zxx = stft(imu_data[:,2], fs=100, nperseg=nperseg, noverlap=noverlap)
    pcm = axes[0, 2].pcolormesh(t, f, np.abs(Zxx)**2, shading='auto')
    # cbar = fig.colorbar(pcm, ax=axes[0, 0], label='Amplitude')
    axes[0, 2].set_title('acc_z')
    pp = np.sum(np.abs(Zxx), axis=0)
    time_vector = np.arange(len(pp)) *((nperseg-noverlap)/fs)
    axes[1, 2].plot(time_vector, pp)
    axes[1, 2].set_title('acc_z')
    acc_z3power = np.sum(abs(Zxx[12:17,:]**2),axis=0)
    acc_z3power[:int(len(acc_z3power)*0.25)] = np.zeros(int(len(acc_z3power)*0.25))
    acc_z3power[-int(len(acc_z3power)*0.05):] = np.zeros(int(len(acc_z3power)*0.05))
    axes[2, 2].plot(time_vector, acc_z3power)
    axes[2, 2].set_title('acc_z')
    f3 = np.sum(abs(Zxx[12:17,:]**2),axis=0)


    # 绘制第4个子图
    f, t, Zxx = stft(imu_data[:,3], fs=100, nperseg=nperseg, noverlap=noverlap)
    pcm = axes[0, 3].pcolormesh(t, f, np.abs(Zxx)**2, shading='auto')
    # cbar = fig.colorbar(pcm, ax=axes[0, 0], label='Amplitude')
    axes[0, 3].set_title('gyro_x')
    pp = np.sum(np.abs(Zxx), axis=0)
    time_vector = np.arange(len(pp)) *((nperseg-noverlap)/fs)
    axes[1, 3].plot(time_vector, pp)
    axes[1, 3].set_title('gyro_x')
    axes[2, 3].plot(time_vector, np.sum(abs(Zxx[12:17,:]**2),axis=0))
    axes[2, 3].set_title('gyro_x')
    f4 = np.sum(abs(Zxx[12:17,:]**2),axis=0)


    # 绘制第5个子图
    f, t, Zxx = stft(imu_data[:,4], fs=100, nperseg=nperseg, noverlap=noverlap)
    pcm = axes[0, 4].pcolormesh(t, f, np.abs(Zxx)**2, shading='auto')
    # cbar = fig.colorbar(pcm, ax=axes[0, 0], label='Amplitude')
    axes[0, 4].set_title('gyro_y')
    pp = np.sum(np.abs(Zxx), axis=0)
    time_vector = np.arange(len(pp)) *((nperseg-noverlap)/fs)
    axes[1, 4].plot(time_vector, pp)
    axes[1, 4].set_title('gyro_y')
    axes[2, 4].plot(time_vector, np.sum(abs(Zxx[12:17,:]**2),axis=0))
    axes[2, 4].set_title('gyro_y')
    f5 = np.sum(abs(Zxx[12:17,:]**2),axis=0)


    # 绘制第6个子图
    f, t, Zxx = stft(imu_data[:,5], fs=100, nperseg=nperseg, noverlap=noverlap)
    pcm = axes[0, 5].pcolormesh(t, f, np.abs(Zxx)**2, shading='auto')
    # cbar = fig.colorbar(pcm, ax=axes[0, 0], label='Amplitude')
    axes[0, 5].set_title('gyro_z')
    pp = np.sum(np.abs(Zxx), axis=0)
    time_vector = np.arange(len(pp)) *((nperseg-noverlap)/fs)
    axes[1, 5].plot(time_vector, pp)
    axes[1, 5].set_title('gyro_z')
    axes[2, 5].plot(time_vector, np.sum(abs(Zxx[12:17,:]**2),axis=0))
    axes[2, 5].set_title('gyro_z')
    f6 = np.sum(abs(Zxx[12:17,:]**2),axis=0)


    # 绘制第7个子图
    f, t, Zxx = stft(imu_data[:,7]-imu_data[:,6], fs=100, nperseg=nperseg, noverlap=noverlap)
    pcm = axes[0, 6].pcolormesh(t, f, np.abs(Zxx)**2, shading='auto')
    # cbar = fig.colorbar(pcm, ax=axes[0, 0], label='Amplitude')
    axes[0, 6].set_title('motor2-1')
    pp = np.sum(np.abs(Zxx), axis=0)
    time_vector = np.arange(len(pp)) *((nperseg-noverlap)/fs)
    axes[1, 6].plot(time_vector, pp)
    axes[1, 6].set_title('motor2-1')
    axes[2, 6].plot(time_vector, np.sum(abs(Zxx[12:17,:]**2),axis=0))
    axes[2, 6].set_title('motor2-1')
    f7 = np.sum(abs(Zxx[12:17,:]**2),axis=0)


    # 绘制第8个子图
    f, t, Zxx = stft(imu_data[:,8]-imu_data[:,6], fs=100, nperseg=nperseg, noverlap=noverlap)
    pcm = axes[0, 7].pcolormesh(t, f, np.abs(Zxx)**2, shading='auto')
    # cbar = fig.colorbar(pcm, ax=axes[0, 0], label='Amplitude')
    axes[0, 7].set_title('motor3-1')
    pp = np.sum(np.abs(Zxx), axis=0)
    time_vector = np.arange(len(pp)) *((nperseg-noverlap)/fs)
    axes[1, 7].plot(time_vector, pp)
    axes[1, 7].set_title('motor3-1')
    axes[2, 7].plot(time_vector, np.sum(abs(Zxx[12:17,:]**2),axis=0))
    axes[2, 7].set_title('motor3-1')
    f8 = np.sum(abs(Zxx[12:17,:]**2),axis=0)


    # 绘制第9个子图
    f, t, Zxx = stft(imu_data[:,9]-imu_data[:,6], fs=100, nperseg=nperseg, noverlap=noverlap)
    pcm = axes[0, 8].pcolormesh(t, f, np.abs(Zxx)**2, shading='auto')
    # cbar = fig.colorbar(pcm, ax=axes[0, 0], label='Amplitude')
    axes[0, 8].set_title('motor4-1')
    pp = np.sum(np.abs(Zxx), axis=0)
    time_vector = np.arange(len(pp)) *((nperseg-noverlap)/fs)
    axes[1, 8].plot(time_vector, pp)
    axes[1, 8].set_title('motor4-1')
    axes[2, 8].plot(time_vector, np.sum(abs(Zxx[12:17,:]**2),axis=0))
    axes[2, 8].set_title('motor4-1')
    f9 = np.sum(abs(Zxx[12:17,:]**2),axis=0)

    plt.close()

    fusion1 = f1*f2*f3
    fusion2 = f4*f5*f6
    fusion3 = f7*f8*f9













    # 原始数据
    x = np.linspace(0-6.25,93.75,int(93.75/6.25)+2)
    y = np.array([-5000, 0, 4485, 7570, 9374, 10885, 12277, 13522, 14691, 15924, 17174, 18179, 19397, 20539, 21692, 22598, 23882])
    


    # 进行曲线拟合
    params, params_covariance = curve_fit(fitting_model_func, x, y)
    # 提取拟合参数
    a = params[0]
    b = params[1]
    c = params[2]
    d = params[3]
    e = params[4]
    x_fit = np.linspace(x.min(), x.max(), 100)
    y_fit = fitting_model_func(x_fit, a, b, c, d, e)

    RPM = fitting_model_func(np.asarray(imu_data[:,6:10]*100), a, b, c, d, e)

    R = np.zeros([imu_data.shape[0], 3, 3])
    for i in range(imu_data.shape[0]):
        R[i, :, :] = rotation_matrix([0,0,0,1])


    m = 0.0362 # mass
    # g = 9.81 # gravity
    g = 9.83 # gravity
    rho = 1.225 # air density
    D = 0.0505  # diameter of the rotor
    rpm2rs = 60 ** 2

    C_t_fixed = 0.09937873049125099
    # C_t_fixed = 0.12037873049125099
    Fa = np.zeros([imu_data.shape[0], 3])
    F = R[:, :, 2]




    for i in range(imu_data.shape[0]):
        F_whole = m * imu_data[i, 0:3]/1000  # G = mg
        # temp[2] += m * g
        r_square = RPM[i, 0]**2 + RPM[i, 1]**2 + RPM[i, 2]**2 + RPM[i, 3]**2
        # r_square = RPM[i, 0]**2*4
        thrust = C_t_fixed / rpm2rs * rho * D ** 4 * r_square # c_t = C_t_fixed / rpm2rs * rho * D ** 4. r_square = n**2+n**2+n**2+n**2
        Fa[i, :] = F_whole - thrust * F[i, :]

    # Offset the decrerase of PWM power manully.
    noisy_signal = Fa[:,2]
    # tem0 = np.linspace(0.008,0.018, int(len(noisy_signal)/2)) # Only for the end half signal 
    # noisy_signal = np.hstack((noisy_signal[:len(noisy_signal)-len(tem0)],noisy_signal[len(noisy_signal)-len(tem0):]+tem0))+0.005
    window_size = 10
    filtered_signal = np.zeros_like(noisy_signal)
    filtered_signal2 = np.zeros_like(filtered_signal)
    for i in range(window_size, len(noisy_signal) - window_size):
        filtered_signal[i] = np.mean(noisy_signal[i - window_size:i])
        filtered_signal2[i] = np.mean(filtered_signal[i - window_size:i])

    # Use first order filtered result for delay consideration
    S1_Target = np.zeros_like(filtered_signal)
    S1_Target[filtered_signal>0.025] = 1 # Easy to show with scaling






    LW_Size = 100
    Guard_Size =15
    PFA = 1e-6
    # PFA = 5e-/2
  
    # fusion_result = np.flipud(fusion_result)
    index, CFAR_Threshold, target_idx = cfar_so(noisy_signal, LW_Size, Guard_Size, PFA)
   
    filtered_signal = np.zeros_like(noisy_signal)
    filtered_signal2 = np.zeros_like(filtered_signal)
    for i in range(window_size, len(noisy_signal) - window_size):
        filtered_signal[i] = np.mean(noisy_signal[i - window_size:i])
        filtered_signal2[i] = np.mean(filtered_signal[i - window_size:i])



    # find the first target threshold before or just arrive the edge
    first0 = np.where(imu_data[:,-1]>600)[0][1]
    last0 = np.where(imu_data[:,-1]>600)[0][-1]

    
    first = np.where(imu_data[first0:last0,-1]<200)[0][1]
    last = np.where(imu_data[first0:last0,-1]<200)[0][-1]
    
    ths_idx = np.where((first-50<target_idx.astype(int)) & (target_idx.astype(int)<last+50))[0]
    if len(ths_idx)>0:
        
        
        th_idx = np.where((first-50<target_idx.astype(int)) & (target_idx.astype(int)<last+50))[0][0]
         
        
        if filtered_signal[th_idx]>0.1:
            f_threshold = np.ones_like(filtered_signal)*0.02
        else:
            f_threshold = np.ones_like(filtered_signal)*filtered_signal[th_idx]
        print(f_threshold)
        print('find\n')
    
    else:

        f_threshold = np.ones_like(filtered_signal)*0.02
        print('not find\n')






   
    # 将三个一维数组合并成一个二维数组: acc_fusion, gyro_fusion, motor_fusion, time, height, 
    merged_arr = np.concatenate((fusion1[:-1], fusion2[:-1], fusion3[:-1], imu_data[:,-2],imu_data[:,-1], noisy_signal, f_threshold))

    # 将合并后的二维数组重塑为三维数组
    fusion_data = merged_arr.reshape((7, -1))

    fusion_data = np.transpose(fusion_data)
    np.savetxt(os.path.join(output_folder, 't1.csv'),fusion_data,delimiter=',')



In [9]:
main()

[-0.00831311 -0.00831311 -0.00831311 ... -0.00831311 -0.00831311
 -0.00831311]
find

